In [1]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

01/19/2021 16:43:51 - INFO - faiss -   Loading faiss.


In [2]:
# Connect to a locally running instance of Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
# document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="ahrq", search_fields='body')
document_store = ElasticsearchDocumentStore(host="localhost", username="elastic", password="VUCpKQjzZvHT81AMDr5Wsg==", index="ahrq_qa")

01/19/2021 16:44:24 - INFO - elasticsearch -   PUT http://localhost:9200/ahrq_qa [status:200 request:0.855s]
01/19/2021 16:44:25 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.847s]


## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store

In this tutorial, we download Wikipedia articles about Game of Thrones, apply a basic cleaning function, and index them in Elasticsearch.

In [4]:
doc_dir = "data/ahrq_txt_files/"

def clean_url_text(text: str) -> str:
    # get rid of multiple new lines
    while "\n\n" in text:
        text = text.replace("\n\n", "\n")

    # remove extremely short lines, combine small paragraphs into larger ones
    lines = text.split("\n")
    cleaned = []
    multi_lines = ''
    for l in lines:
        if len(l) > 100:
            multi_lines += l + '\n\t'
        if len(l) > 500:
            cleaned.append(multi_lines)
            multi_lines = ''
    
    if multi_lines: cleaned.append(multi_lines) 
    text = "\n\n".join(cleaned)

    # add paragraphs (recognized by double new line)
    # text = text.replace("\n", "\n\n")

    # remove empty paragrahps
    # text = re.sub(r"(==.*==\n\n\n)", "", text)

    return text

from elasticsearch import Elasticsearch
import json
# Loop through ES and save all docs as txt files so we can use convert_files_to_dict funciton

host = 'localhost'
port = 9200
client = Elasticsearch(f'{host}:{port}', http_auth=('elastic', 'VUCpKQjzZvHT81AMDr5Wsg=='))
index = 'preprocessed_webpages'

match_all = {
    "size": 100,
    "query": {
        "match_all": {}
    }
}

resp = client.search(
    index = index,
    body = match_all,
    scroll = '2s' # length of time to keep search context
)

old_scroll_id = resp['_scroll_id']

dicts = []
while len(resp['hits']['hits']):
    resp = client.scroll(
        scroll_id = old_scroll_id,
        scroll = '2s' # length of time to keep search context
    )

    # check if there's a new scroll ID
    if old_scroll_id != resp['_scroll_id']:
        print ("NEW SCROLL ID:", resp['_scroll_id'])

    # keep track of pass scroll _id
    old_scroll_id = resp['_scroll_id']
    
    for doc in resp['hits']['hits']:
        del doc['_source']['tags']
        dicts.append(doc['_source'])




12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/ahrq_annotated/_search?scroll=2s [status:200 request:0.063s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.038s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.044s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.036s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.032s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.017s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.034s]
12/23/2020 09:48:37 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [sta

12/23/2020 09:48:39 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.024s]
12/23/2020 09:48:39 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.012s]
12/23/2020 09:48:39 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll?scroll=2s [status:200 request:0.003s]


Write Data to files and ES index.

In [ ]:
# Save docs as txt files
import os
if not os.path.exists(doc_dir):
    os.makedirs(doc_dir)
    
    for d in dicts:
        with open(doc_dir + d['url'][8:].replace('/', 'x_!_x') + '.txt', 'w', encoding="utf-8") as f:
            f.write(d['body'])
            
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_url_text, split_paragraphs=True)

from pprint import pprint
for d in dicts[:3]:
    pprint(d)

for d in dicts:
    d['meta']['url'] = 'https://' + d['meta']['name'].replace('x_!_x', '/')[:-4]
    del d['meta']['name']

s = sum([len(d['text']) for d in dicts])
print('average length of paragraph: ', s/len(dicts))

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

In [11]:
cleaned_dicts

[{'text': 'This document outlines measure specifications for the ABCS of cardiovascular disease prevention (aspirin use, blood pressure control, cholesterol management, and smoking cessation) that were used in EvidenceNOW. \n\tAcknowledgements:  EvidenceNOW Southwest, Agency for Healthcare Research and Quality EvidenceNOW Initiative\n\t',
  'meta': {'name': 'www.ahrq.govx_!_xevidencenowx_!_xresultsx_!_xresearchx_!_xmeasures-specifications-sw.html.txt'}},
 {'text': 'USPSTF Publishes Recommendation Statement on Interventions to Prevent Illicit Drug Use [May 26, 2020] \n\tThe USPSTF published a final recommendation statement and evidence review on  Primary Care-Based Interventions to Prevent Illicit Drug Use in Children, Adolescents, and Young Adults  on May 26, 2020. The Task Force concludes that the current evidence is insufficient to assess the balance of benefits and harms of primary care-based behavioral counseling interventions to prevent illicit drug use, including nonmedical use o

## Initalize Retriever, Reader,  & Finder

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

#### FARMReader

### Finder

The Finder sticks together reader and retriever in a pipeline to answer our actual questions. 

In [14]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.retriever.sparse import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

finder = Finder(reader, retriever)

12/11/2020 18:02:21 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/11/2020 18:02:21 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/11/2020 18:02:24 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/11/2020 18:02:28 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/11/2020 18:02:28 - INFO - farm.inf

## Voilà! Ask a question!

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
question = "What department is AHRQ a part of?"
prediction = finder.get_answers(question, top_k_retriever=10, top_k_reader=5)

print_answers(prediction, details="medium")

12/11/2020 18:02:28 - INFO - elasticsearch -   POST http://localhost:9200/ahrq_qa/_search [status:200 request:0.053s]
12/11/2020 18:02:28 - INFO - haystack.finder -   Got 10 candidates from retriever
12/11/2020 18:02:28 - INFO - haystack.finder -   Reader is looking for detailed answer in 11648 chars ...


In [62]:
objs = finder.retriever.retrieve(question, filters=None, top_k=10, index="ahrq")
dicts = [o.to_dict() for o in objs]
dicts[0].keys()

11/10/2020 18:15:35 - INFO - elasticsearch -   POST http://localhost:9200/ahrq/_search [status:200 request:0.014s]
11/10/2020 18:15:35 - INFO - haystack.retriever.sparse -   Got 10 candidates from retriever


In [61]:
d = dicts[0:1]
reader.predict(question, objs, 1)

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 43.49 Batches/s]


{'question': 'What department is AHRQ a part of?',
 'no_ans_gap': 12.131163120269775,
 'answers': [{'answer': 'Health and Human Services',
   'score': 14.955009460449219,
   'probability': 0.866386080089388,
   'context': 'nd Quality (AHRQ). As part of the budget for the Department of Health and Human Services (HHS), the performance budget submission both provides backgr',
   'offset_start': 63,
   'offset_end': 88,
   'offset_start_in_doc': 821,
   'offset_end_in_doc': 846,
   'document_id': '929e7758-0c10-43ac-bf4f-3efd4645d700'}]}

In [28]:
type(reader.inferencer.processor)

farm.data_handler.processor.SquadProcessor

type(reader.inferencer.model.language_model.model)

In [53]:
a = reader.inferencer.model.prediction_heads
print(type(a))
if len(a) > 0:
    print(a[0].aggregate_preds)
print(type(a[0]))
del a

<class 'torch.nn.modules.container.ModuleList'>
<bound method QuestionAnsweringHead.aggregate_preds of QuestionAnsweringHead(
  (feed_forward): FeedForwardBlock(
    (feed_forward): Sequential(
      (0): Linear(in_features=768, out_features=2, bias=True)
    )
  )
)>
<class 'farm.modeling.prediction_head.QuestionAnsweringHead'>


In [20]:
type(reader.inferencer.model.language_model.model)

transformers.modeling_roberta.RobertaModel

In [19]:
dir(reader.inferencer.model.language_model.model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_convert_head_mask_to_5d',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_forward_unimplemented',
 '_generate_beam_search',
 '_generate_no_beam_search',
 '_get_name',
 '_get_resized_embeddings',
 '_hook_rss_memory_post_forward',
 '_hook_rss_memory_pre_forward',
 '_init_weights',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_prune_heads',
 '_register_load_state_dict_pre_hook',
 '_register_st